In [1]:
from sqlalchemy import create_engine
from datetime import date
import pandas as pd

In [2]:
user = 'weiklikfhjocec'
password = '5e1d65ca00df8aa20d5c17cff300e2e881042dc68df4d26fa898f84f833c9e87'
host = 'ec2-18-210-64-223.compute-1.amazonaws.com'
port = 5432
database = 'd2cneae9rn5fru'

In [3]:
def get_connection():
    return create_engine(
        url="postgresql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )

def test_connection():
    try:
        # GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
        engine = get_connection()
        print(f"Connection to the {host} for user {user} created successfully.")
    except Exception as ex:
        print("Connection could not be made due to the following error: \n", ex)
    
    return engine

def get_data(engine, sql):
    dfs=[]
    for chunk in pd.read_sql_query(sql,con=engine, chunksize=2):
        dfs.append(chunk)
    df = pd.concat(dfs, ignore_index=True)
    return df

In [4]:
myengine = test_connection()

Connection to the ec2-18-210-64-223.compute-1.amazonaws.com for user weiklikfhjocec created successfully.


In [5]:
patient_sql = '''SELECT * FROM patients'''
enrollment_sql = '''SELECT * FROM enrollments'''
discharge_sql = '''SELECT * FROM discharges OFFSET 2'''

In [6]:
discharge_df = get_data(myengine, discharge_sql)
enrollment_df = get_data(myengine, enrollment_sql)
patient_df = get_data(myengine, patient_sql)

## Write discharge, enrollment & patient data into an Excel file

In [9]:
day = date.today().strftime("%d/%b/%Y")
filename = 'Data Collection Summary-'+day+'.xlsx'

In [11]:
patientID_df = patient_df[['id','patient_id']]

In [12]:
enrollment_join = pd.merge(patientID_df, 
                      enrollment_df, 
                      left_on ='id',
                      right_on='patient_id')

discharge_join = pd.merge(patientID_df, 
                      discharge_df, 
                      left_on ='id',
                      right_on='patient_id')

In [13]:
discharge_join.drop(['id_x', 'id_y','patient_id_y'], axis=1, inplace=True)
discharge_join.rename({'patient_id_x': 'Patient ID'}, axis=1, inplace=True)

In [14]:
enrollment_join.drop(['id_x','id_y','patient_id_y'], axis=1, inplace=True)
enrollment_join.rename({'patient_id_x': 'Patient ID'}, axis=1, inplace=True)

In [21]:
enrollment_join.sort_values(by = ['Patient ID','fever_day'], ascending=[True, True], inplace=True)

In [23]:
patient_df.drop(['id'], axis=1, inplace=True)
patient_df.rename({'patient_id': 'Patient ID'}, axis=1, inplace=True)

In [24]:
patient_df['updatedAt'] = patient_df['updatedAt'].dt.tz_localize(None)
enrollment_join['updatedAt'] = enrollment_join['updatedAt'].dt.tz_localize(None)
discharge_join['updatedAt'] = discharge_join['updatedAt'].dt.tz_localize(None)
discharge_join['datetime_death_transfer'] = discharge_join['datetime_death_transfer'].dt.tz_localize(None)
discharge_join['ns1_date_1'] = discharge_join['ns1_date_1'].apply(lambda x:str(x))
discharge_join['ns1_date_2'] = discharge_join['ns1_date_2'].apply(lambda x:str(x))
discharge_join['igm_date_1'] = discharge_join['igm_date_1'].apply(lambda x:str(x))
discharge_join['igm_date_2'] = discharge_join['igm_date_2'].apply(lambda x:str(x))
discharge_join['igg_date_1'] = discharge_join['igg_date_1'].apply(lambda x:str(x))
discharge_join['igg_date_2'] = discharge_join['igg_date_2'].apply(lambda x:str(x))
discharge_join['chest_x_ray_datetime'] = discharge_join['chest_x_ray_datetime'].apply(lambda x:str(x))
discharge_join['ultrasound_datetime'] = discharge_join['ultrasound_datetime'].apply(lambda x:str(x))

In [38]:
day = date.today().strftime("%d/%b/%Y")
# filename = 'Data Collection Summary-'+day+'.xlsx'
filename = 'Data_Collection_Summary.xlsx'
with pd.ExcelWriter(filename) as writer:
    patient_df.to_excel(writer, sheet_name="Admission Clerking Form", index=False)
    enrollment_join.to_excel(writer, sheet_name="Daily Review Form", index=False)
    discharge_join.to_excel(writer, sheet_name="Discharge Form", index=False)